In [ ]:
%pip install mediapipe opencv-python


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image


mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hol = mp.solutions.holistic

In [3]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
   while cap.isOpened():
      ret, frame = cap.read()

      '''
      the cv2 library uses bgr by default but mediapipe wants 
      frames in rgb format
      '''
      
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      # more memory efficient because ensures that we don't need to make a copy of our image (array)
      image.flags.writeable = False 


      results = pose.process(image)

      
      
      
      # TODO: get cool custom font
      # font = ImageFont.truetype("Roboto-Regular.ttf", 50)



      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      #  prepare text renderer
      pil_im = Image.fromarray(image)
      draw = ImageDraw.Draw(pil_im)
      font = ImageFont.load_default()

      try:
         landmarks = results.pose_landmarks.landmark
         
         shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
         elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
         wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

         angle = calculate_angles(shoulder, elbow, wrist)
         location_for_text = tuple(np.multiply(elbow, [640,480]).astype(int))
         draw.text(location_for_text, str(angle), font=font)

         # cv2.putText(
         #    image,
         #    str(angle),
         #    #multiple by 640 and 480 to normalize it to display to the screen
         #    tuple(np.multiply(elbow, [640,480]).astype(int)),
         #    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.line_AA
         # )
      except:
         pass



      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

      cv2.imshow('Feed', image)

      key = cv2.waitKey(10)

      if key==ord('q'):
         cv2.destroyAllWindows()
         break

   cv2.waitKey(1)
   cap.release()
   cv2.destroyAllWindows()




2024-08-24 22:01:23.213 Python[70738:33283962] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1724562084.448101 33283962 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1724562084.564792 33285420 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724562084.573431 33285420 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/vikrantverma/Documents/mediapipe gym tracker/myenv/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() ins

In [2]:
def calculate_angles(start, mid, end):
   start = np.array(start)
   mid = np.array(mid)
   end = np.array(end)

   radians = np.arctan2(end[1] - mid[1], end[0] - mid[0]) - np.arctan2(start[1] - mid[1], start[0] - mid[0])
   angle = np.abs(radians*180.0/np.pi)

   if angle > 180:
      angle = 360 - angle
   
   return angle

